Holaaaaaaaa

In [4]:
!pip install pandas

     ---------------------------------------- 11.6/11.6 MB 1.4 MB/s eta 0:00:00
     ------------------------------------ 505.5/505.5 KB 990.5 kB/s eta 0:00:00
     ---------------------------------------- 12.9/12.9 MB 1.8 MB/s eta 0:00:00
     -------------------------------------- 345.4/345.4 KB 2.1 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\delva\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
import pandas as pd